<a href="https://colab.research.google.com/github/jotalian/projeto_inicial/blob/main/rifa_beneficente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q -U google-generativeai

import pathlib
import textwrap

import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-pro')

In [3]:
!pip install rich -q -U
!pip install qgrid -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 889.2/889.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00


In [ ]:
import random
from rich.table import Table
from rich.console import Console

class Rifa:
    """
    Classe para gerenciar uma rifa.
    """

    def __init__(self, nome, valor_rifa, total_numeros, premio):
        """
        Inicializa uma nova rifa.

        Args:
            nome (str): Nome da rifa.
            valor_rifa (float): Valor de cada número.
            total_numeros (int): Total de números da rifa.
            premio (str): Prêmio da rifa.
        """
        self.nome = nome
        self.valor_rifa = valor_rifa
        self.total_numeros = total_numeros
        self.premio = premio
        self.numeros_vendidos = {}
        self.total_arrecadado = 0.00
        self.numeros_disponiveis = list(range(1, total_numeros + 1))
        self.historico_dezenas = []

    def comprar_numero(self, nome_comprador, numero):
        """
        Permite a compra de um número da rifa.

        Args:
            nome_comprador (str): Nome do comprador do número.
            numero (int): Número da rifa a ser comprado.

        Returns:
            bool: True se a compra foi bem sucedida, False caso contrário.
        """
        if not self._validar_numero(numero):
            return False
        if numero in self.numeros_vendidos:
            print(f"O número {numero:0{2 if numero < 100 else 3}} já foi escolhido para {self.numeros_vendidos[numero]}. Por favor, escolha outro.")
            return False
        self.numeros_vendidos[numero] = nome_comprador
        self.numeros_disponiveis.remove(numero)
        self.total_arrecadado += self.valor_rifa
        dezena = (numero - 1) // 10 + 1
        if dezena not in self.historico_dezenas:
            self.historico_dezenas.append(dezena)
        print(f"Parabéns {nome_comprador}! Você adquiriu o número {numero:0{2 if numero < 100 else 3}} da {self.nome}.")
        return True

    def realizar_sorteio(self):
        """
        Realiza o sorteio da rifa e retorna o número vencedor.

        Returns:
            int: Número sorteado vencedor da rifa.
        """
        numero_sorteado = random.choice(list(self.numeros_vendidos.keys()))
        print(f"\nE o número sorteado é... {numero_sorteado:0{2 if numero_sorteado < 100 else 3}}!")
        return numero_sorteado

    def verificar_vencedor(self, numero_sorteado):
        """
        Verifica e imprime o vencedor do sorteio.

        Args:
            numero_sorteado (int): Número vencedor do sorteio.
        """
        vencedor = self.numeros_vendidos.get(numero_sorteado)
        print(f"\nParabéns {vencedor}, você ganhou {self.premio}!")

    def _validar_numero(self, numero):
        """
        Valida se o número está dentro do intervalo válido (1 a total_numeros).

        Args:
            numero (int): Número a ser validado.

        Returns:
            bool: True se o número for válido, False caso contrário.
        """
        if numero <= 0 or numero > self.total_numeros:
            print("Por favor, escolha um número entre 1 e {}.".format(self.total_numeros))
            return False
        return True

    def _validar_nome(self, nome):
        """
        Valida se o nome contém apenas letras e espaços.

        Args:
            nome (str): Nome a ser validado.

        Returns:
            bool: True se o nome for válido, False caso contrário.
        """
        if all(c.isalpha() or c.isspace() for c in nome):
            return True
        else:
            print("Por favor, digite apenas letras no campo nome.")
            return False

    def exibir_tabela_vendidos(self):
        """Exibe a tabela de números vendidos com nome e número usando rich."""
        table = Table(title="Números Vendidos")
        table.add_column("Nome", style="cyan", width=16)
        table.add_column("Número", style="magenta", width=5, justify="right")
        for numero, nome in self.numeros_vendidos.items():
            table.add_row(nome, f"{numero:0{2 if numero < 100 else 3}}")
        console = Console()
        console.print(table)

    def exibir_numeros_disponiveis(self):
        """Exibe os números disponíveis em uma tabela com 5 colunas usando rich."""
        table = Table(title="Números Disponíveis")
        for i in range(5):
            table.add_column(f"Coluna {i+1}", style="green", width=5, justify="right")
        for i in range(0, len(self.numeros_disponiveis), 5):
            row = []
            for j in range(5):
                if i + j < len(self.numeros_disponiveis):
                    row.append(f"{self.numeros_disponiveis[i + j]:0{2 if self.numeros_disponiveis[i + j] < 100 else 3}}")
                else:
                    row.append("")
            table.add_row(*row)
        console = Console()
        console.print(table)

# --- Configurações da rifa ---
nome_rifa = "Rifa Beneficente para Lu, ajude uma amiga"
valor_rifa = 10.00  # Valor de cada número
total_numeros = 100  # Total de números da rifa
premio = "1 (Um) Carneiro!"  # Prêmio da rifa

# --- Lógica da rifa ---
while True:
    rifa = Rifa(nome_rifa, valor_rifa, total_numeros, premio)
    print(f"\n--- {rifa.nome} ---")
    pausado = False
    while len(rifa.numeros_disponiveis) > 0:
        if not pausado:
            rifa.exibir_numeros_disponiveis()
            while True:
                nome = input("\nDigite seu nome (ou 'PARAR' para pausar): ")
                if nome.upper() == "PARAR":
                    print("Escolha de números pausada. A rifa será finalizada quando todos os números forem confirmados.")
                    pausado = True
                    break
                elif rifa._validar_nome(nome):
                    nome = nome[:50]  # Limita o nome a 50 caracteres
                    break  # Nome válido
        if pausado:
            continue
        while True:
            numeros_escolhidos_str = input("Escolha um ou mais números da lista acima, separados por espaço: ")
            if all(c.isdigit() or c.isspace() for c in numeros_escolhidos_str):
                numeros_escolhidos = [int(n) for n in numeros_escolhidos_str.split()]
                if all(rifa._validar_numero(n) for n in numeros_escolhidos):
                    break  # Números válidos
                else:
                    print("Por favor, escolha números entre 1 e {}.".format(rifa.total_numeros))
            else:
                print("Por favor, digite apenas números no campo número.")
        for i, numero in enumerate(numeros_escolhidos):
            if rifa.comprar_numero(nome, numero):
                print(f"Ainda restam {len(rifa.numeros_disponiveis)} números.")
                rifa.exibir_tabela_vendidos()
            else:
                if i < len(numeros_escolhidos) - 1:
                    print(f"O próximo número ({numeros_escolhidos[i+1]:0{2 if numeros_escolhidos[i+1] < 100 else 3}}) será atribuído a {nome}.")
        else:
            pausado = False
    # Realiza o sorteio após todos os números serem vendidos
    numero_sorteado = rifa.realizar_sorteio()
    rifa.verificar_vencedor(numero_sorteado)
    print(f"\nO total arrecadado foi R${rifa.total_arrecadado:.2f}. Obrigado a todos pela participação!")
    print("\nHistórico de dezenas escolhidas nesta rifa:", rifa.historico_dezenas)
    if input("\nDeseja realizar outra rifa? (s/n): ").lower() != 's':
        break